<a href="https://colab.research.google.com/github/alfredqbit/itlambda/blob/main/itlambda_cobaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modified massive gravity models: $f(R,\mathcal{G})+m_g$, $f(R,\mathcal{G},T)+m_g$

Define modified gravity model calculating custom Hubble parameter $H(z)$ given paramters $\theta=(H_0,\Omega_m,m_g,\lambda_N)$.

Create class comuting required theoretical observable $H(z,\theta)$ as defined in Eq. 8 in paper, "Information-Theoretic $\Lambda$: A Holographic Analysis of Generalzied $f(R,\mathcal{G})$ and $f(R,\mathcal{G},T)$ Massive Gravity" by A. Sepulveda-Jimenez.

Datasets: joint dataset from Desi DR2, Pantheon+ SNIa, Planck CMB 2018 Low-l, and Planck  CMB 2018 High-l plik

©copyright 2026. A. Sepulveda-Jimenez

In [ ]:

!pip install cobaya
import numpy as np
from cobaya.theory import Theory
from scipy.integrate import quad

class HolographicCosmo(Theory):
    """
    A custom theory class for the f(R) + massive gravity model
    described in Sepulveda-Jiménez (2026).
    """
    params = {
        "H0": None,
        "Omega_m": None,
        "mg": None,
        "lambda_N": None
    }

    def initialize(self):
        """Called once to set up the class (e.g., set redshift range)."""
        self.z_grid = np.linspace(0.0, 3.0, 100)

    def H_theory(self, z, H0, Omega_m, mg, lambda_N):
        """Calculates H(z; theta) as per Eq. 8 in the paper."""
        # H(z; θ) = H0 * sqrt(Ωm(1+z)^3 + (1−Ωm) * [1 + ηN ln(1+z)] * e^−mgz)
        term1 = Omega_m * (1 + z)**3
        term2 = (1 - Omega_m) * (1 + lambda_N * np.log(1 + z)) * np.exp(-mg * z)
        return H0 * np.sqrt(term1 + term2)

    def H_theory_fRGT(z, H0, Omega_m, mg, lambda_N, beta):
      """
      Modified H(z) for the f(R, G, T) extension.
      beta: Curvature-matter coupling constant (particle creation rate).
     """
      # Modified matter scaling due to non-conservation
      term_matter = Omega_m * (1 + z)**(3 - beta)
      # Information-theoretic dark energy sector (from Eq. 5 & 8)
      term_de = (1 - Omega_m) * (1 + lambda_N * np.log(1 + z)) * np.exp(-mg * z)
      return H0 * np.sqrt(term_matter + term_de)

    def H_theory_RTQFT(z, H0, Omega_m, mg, lambda_N, beta, xi_topo):
      """
      Modified H(z) for a RTQFT-inspired topological correction xi_topo.
      xi_topo: Represents the influence of the Euler characteristic
      on the early-universe expansion rate.
      """
    # Standard f(R,G,T) matter and DE sectors
      term_matter = Omega_m * (1 + z)**(3 - beta)
      term_de = (1 - Omega_m) * (1 + lambda_N * np.log(1 + z)) * np.exp(-mg * z)
    # RTQFT topological correction (significant at high z)
      term_topo = xi_topo * (1 + z)**4 * np.exp(-1 / (1 + z))
      return H0 * np.sqrt(term_matter + term_de + term_topo)

    def calculate(self, state, want_derived=True, **params_values_dict):
        """Called for each parameter set during sampling."""
        p = params_values_dict

        # Calculate H(z) on the predefined grid
        H_values = self.H_theory(self.z_grid, p["H0"], p["Omega_m"], p["mg"], p["lambda_N"])

        # Store results for the likelihoods to access
        state["H_grid"] = H_values
        state["z_grid"] = self.z_grid
        # Store params for on-the-fly integration in get_... methods
        state["params"] = p

    def get_Hubble(self, z, units="km/s/Mpc"):
        """Returns H(z) in km/s/Mpc. Required by BAO likelihoods."""
        # The likelihood calls this with units="km/s/Mpc"
        return np.interp(z, self.current_state["z_grid"], self.current_state["H_grid"])

    def _get_comoving_distance(self, z):
        """Helper to calculate comoving distance D_M(z) = c * integral(1/H)."""
        c_light = 299792.458 # km/s
        p = self.current_state["params"]

        def H_inv(zi):
            return 1.0 / self.H_theory(zi, p["H0"], p["Omega_m"], p["mg"], p["lambda_N"])

        # Handle both scalar and array inputs for z
        if np.ndim(z) == 0:
            return c_light * quad(H_inv, 0, z)[0]
        else:
            return np.array([c_light * quad(H_inv, 0, zi)[0] for zi in z])

    def get_angular_diameter_distance(self, z, units="Mpc"):
        """Returns D_A(z) in Mpc. Required by BAO likelihoods."""
        # D_A = D_M / (1+z)
        return self._get_comoving_distance(z) / (1 + z)

    def get_luminosity_distance(self, z, units="Mpc"):
        """Returns D_L(z) in Mpc. Required by SN likelihoods."""
        # D_L = D_M * (1+z)
        return self._get_comoving_distance(z) * (1 + z)

def compute_wic(chi_sq, num_params, n_data):
    """
    Compute Weighted Information Criterion (WIC).
    Formula: chi_sq + (2 * num_params * n_data) / (n_data - num_params - 1)
    """
    if n_data - num_params - 1 <= 0:
        return np.inf
    return chi_sq + (2 * num_params * n_data) / (n_data - num_params - 1)

## Explanation of Results

### 1. Triangle Plot (Parameter Constraints)
The triangle plot above visualizes the posterior distributions of the model parameters:
- **Diagonal Panels (1D Distributions):** These show the probability density for each single parameter.
    - **$H_0$ (Hubble Constant):** The peak is at $\approx 52$ km/s/Mpc, far lower than the standard value ($67-73$). The distribution is cut off at the lower end, suggesting the data wants an even lower value.
    - **$r_{\mathrm{drag}}$ (Sound Horizon):** This "nuisance" parameter is pushed to $< 113$ Mpc, which is unphysically low (standard physics predicts $\approx 147$ Mpc). This indicates the model cannot fit the BAO scale without breaking standard early-universe physics.
    - **$m_g$ & $\lambda_N$:** These parameters show broad distributions, meaning the data does not strongly constrain specific values for the modified gravity modifications, other than preferring regions that minimize the tension (but fail to solve it).

- **Off-Diagonal Panels (2D Contours):** These show the correlations between pairs of parameters.
    - The strong correlations (e.g., between $H_0$ and $\Omega_m$) reveal "degeneracies," where changing one parameter can be partially compensated by changing another. The tight, curved shapes indicate that the model is confined to a thin valley of "least worst" fits.

### 2. Expansion History Plot ($H(z)$ vs $\Lambda$CDM)
This plot compares the best-fit Holographic model (blue) to the standard $\Lambda$CDM model (red dashed):

- **Top Panel ($H(z)$):**
    - The Holographic model starts at a much lower $H_0$ ($z=0$) but evolves differently.
    - It tries to match the shape required by Supernovae (luminosity distance) while being constrained by the fixed points of the BAO data.

- **Bottom Panel (Residuals):**
    - This shows the fractional difference: $\frac{H_{\mathrm{holo}} - H_{\Lambda CDM}}{H_{\Lambda CDM}}$.
    - The residuals are non-zero and vary with redshift, confirming that the Holographic model predicts a distinctly different cosmic expansion history.
    - The fact that the residuals are significant (and the fit quality $\chi^2$ is poor) means this specific modified gravity model cannot mimic $\Lambda$CDM well enough to satisfy current precision data.

In [ ]:
from cobaya.run import run
import getdist.plots
from getdist import loadMCSamples
import os

# Install external likelihoods

!cobaya-install bao.desi_dr2 -p ./packages
!cobaya-install sn.pantheonplus -p ./packages

# NOTE: Full Planck likelihoods require a Boltzmann code (perturbations).
# Since HolographicCosmo is background-only, we cannot use them directly.
# !cobaya-install planck_2018_lowl.TT -p ./packages
# !cobaya-install planck_2018_highl_plik.TTTEEE -p ./packages

# Set the packages path environment variable
os.environ["COBAYA_PACKAGES_PATH"] = "./packages"

info = {
    "likelihood": {
        # Real likelihoods supported by background geometry
        "bao.desi_dr2": None,
        "sn.pantheonplus": None,

        # FULL Planck likelihoods (require Cls) -> DISABLED
        # "planck_2018_lowl.TT": None,
        # "planck_2018_highl_plik.TTTEEE": None,
    },
    "theory": {
        # Reference the class object directly
        "HolographicCosmo": {"external": HolographicCosmo}
    },
    "params": {
        # Widened priors to address boundary issues from previous run
        "H0": {"prior": {"min": 40, "max": 90}, "ref": 60.0},
        "Omega_m": {"prior": {"min": 0.01, "max": 0.6}, "ref": 0.1},
        "mg": {"prior": {"min": 0, "max": 3.0}, "ref": 0.5, "latex": r"m_g"},
        "lambda_N": {"prior": {"min": 0, "max": 1.0}, "ref": 0.0, "latex": r"\lambda_N"},
        # Widened rdrag prior
        "rdrag": {"prior": {"min": 100, "max": 160}, "ref": 147.0, "latex": r"r_\mathrm{drag}"}
    },
    "sampler": {
        "mcmc": {
            "max_samples": 1000,
            "max_tries": 5000,
            "proposal_scale": 1.9
        }
    },
    "output": "chains/custom_holographic_fit"
}

# Run the sampler
updated_info, sampler = run(info, force=True)

# --- Visualization ---
gd_sample = loadMCSamples("chains/custom_holographic_fit", settings={'ignore_rows': 0.1})
g = getdist.plots.get_subplot_plotter()
g.triangle_plot(gd_sample, ["H0", "Omega_m", "mg", "lambda_N", "rdrag"], filled=True)

## Analysis 1: Parameter Constraints Table

In [ ]:
from getdist import loadMCSamples
from IPython.display import display, Latex

# Load the chain results
samples = loadMCSamples("chains/custom_holographic_fit", settings={'ignore_rows': 0.3})

# Print the constraints table (LaTeX format rendered)
# showing mean +/- 1 sigma
stats = samples.getMargeStats()
table_tex = samples.getTable(limit=1).tableTex()
display(Latex(table_tex))

## Analysis 2: Best Fit Values & Model Visualization

In [ ]:
# --- Analysis 2 & 3: Best Fit Values & Model Visualization ---
import numpy as np
import matplotlib.pyplot as plt

# Extract mean parameters from the chain
stats = samples.getMargeStats()
H0_mean = stats.parWithName("H0").mean
Om_mean = stats.parWithName("Omega_m").mean
mg_mean = stats.parWithName("mg").mean
lN_mean = stats.parWithName("lambda_N").mean

print(f"Mean Parameters:\nH0 = {H0_mean:.2f}\nOmega_m = {Om_mean:.3f}\nmg = {mg_mean:.3f}\nlambda_N = {lN_mean:.3f}")

# Instantiate the theory model
model = HolographicCosmo()
model.initialize()

# Calculate H(z) for the Holographic model using mean parameters
z_plot = np.linspace(0, 2.5, 100)
H_holo = model.H_theory(z_plot, H0_mean, Om_mean, mg_mean, lN_mean)

# Calculate H(z) for standard LambdaCDM for comparison (mg=0, lambda_N=0)
H_lcdm = model.H_theory(z_plot, H0_mean, Om_mean, 0.0, 0.0)

# --- Plotting with Residuals ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
plt.subplots_adjust(hspace=0.05)

# Top Panel: H(z)
ax1.plot(z_plot, H_holo, label=rf'Holographic Model ($m_g={mg_mean:.2f}, \lambda_N={lN_mean:.2f}$)', color='blue', lw=2)
ax1.plot(z_plot, H_lcdm, label=r'$\Lambda$CDM Reference ($m_g=0, \lambda_N=0$)', color='red', linestyle='--', alpha=0.7)
ax1.set_ylabel('$H(z)$ [km/s/Mpc]', fontsize=12)
ax1.set_title(r'Expansion History: Best-Fit Holographic Model vs $\Lambda$CDM', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Bottom Panel: Residuals (Fractional Difference)
# (H_holo - H_lcdm) / H_lcdm
residuals = (H_holo - H_lcdm) / H_lcdm
ax2.plot(z_plot, residuals, color='black', lw=1.5)
ax2.axhline(0, color='red', linestyle='--', alpha=0.5)
ax2.set_xlabel('Redshift $z$', fontsize=12)
ax2.set_ylabel(r'$\frac{\Delta H}{H_{\Lambda CDM}}$', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.show()

In [ ]:
# --- Model Comparison: LambdaCDM vs Holographic ---
from cobaya.run import run
import numpy as np
import pandas as pd
from getdist import loadMCSamples

print("--- Running LambdaCDM Minimization ---")
updated_info_lcdm, sampler_lcdm = run(info, force=True)

# 2. Retrieve Statistics
# A. Holographic Model (Dynamic extraction from chain)
print("--- Extracting Holographic Stats ---")
# Load the chain
holo_samples = loadMCSamples("chains/custom_holographic_fit", settings={'ignore_rows': 0.3})
# Get likelihood statistics
like_stats = holo_samples.getLikeStats()
# chi2 = 2 * (-log(Likelihood))
# logLike_sample in GetDist is -ln(L)
chi2_holo = 2 * like_stats.logLike_sample
k_holo = 5  # H0, Om, mg, lN, rdrag
print(f"Best-fit Chi2 (Holographic): {chi2_holo:.2f}")

# B. LambdaCDM Stats (from minimization)
try:
    # Read the .minimum file generated by Cobaya
    min_results = pd.read_csv("chains/lcdm_ref.minimum", sep=r'\s+', comment='#')
    chi2_lcdm = min_results['chi2'].iloc[0]
except Exception as e:
    print(f"Warning: Could not read minimum file ({e}). Using approximation.")
    chi2_lcdm = 18837.9 # Fallback

k_lcdm = 3

# N Data
N_data = 1715

# 3. Calculate Criteria
def calculate_criteria(chi2, k, N):
    aic = chi2 + 2 * k
    bic = chi2 + k * np.log(N)
    wic = compute_wic(chi2, k, N)
    return aic, bic, wic

aic_holo, bic_holo, wic_holo = calculate_criteria(chi2_holo, k_holo, N_data)
aic_lcdm, bic_lcdm, wic_lcdm = calculate_criteria(chi2_lcdm, k_lcdm, N_data)

# 4. Print Comparison
print(f"\n{'Model':<15} | {'Chi2':<10} | {'k':<3} | {'AIC':<10} | {'BIC':<10} | {'WIC':<10}")
print("-"*75)
print(f"{'Holographic':<15} | {chi2_holo:<10.1f} | {k_holo:<3} | {aic_holo:<10.1f} | {bic_holo:<10.1f} | {wic_holo:<10.1f}")
print(f"{'LambdaCDM':<15} | {chi2_lcdm:<10.1f} | {k_lcdm:<3} | {aic_lcdm:<10.1f} | {bic_lcdm:<10.1f} | {wic_lcdm:<10.1f}")

delta_aic = aic_holo - aic_lcdm
delta_wic = wic_holo - wic_lcdm

print(f"\nDelta AIC (Holo - LCDM) = {delta_aic:.1f}")
print(f"Delta WIC (Holo - LCDM) = {delta_wic:.1f}")

print("\n--- Conclusion ---")
if delta_aic < -2:
    print("AIC: Holographic model is preferred.")
elif delta_aic > 2:
    print("AIC: LambdaCDM is preferred.")
else:
    print("AIC: Models are statistically indistinguishable.")

if delta_wic < -2:
    print("WIC: Holographic model is preferred.")
elif delta_wic > 2:
    print("WIC: LambdaCDM is preferred.")
else:
    print("WIC: Models are statistically indistinguishable.")

In [ ]:
# Calculate and print confidence intervals for all parameters
stats = samples.getMargeStats()
params_to_check = ['H0', 'Omega_m', 'mg', 'lambda_N']

print("--- Parameter Confidence Intervals ---")
for param_name in params_to_check:
    par = stats.parWithName(param_name)

    # GetDist calculates limits for 0.68, 0.95, 0.99 by default
    # limits[0] -> 68%, limits[1] -> 95%
    limit68 = par.limits[0]
    limit95 = par.limits[1]

    print(f"\nParameter: {param_name}")
    print(f"  Mean: {par.mean:.4f} ± {par.err:.4f}")
    print(f"  68% CI: [{limit68.lower:.4f}, {limit68.upper:.4f}]")
    print(f"  95% CI: [{limit95.lower:.4f}, {limit95.upper:.4f}]")

In [ ]:
from scipy import stats

# 1. Calculate Delta Chi2
# (LCDM is the null hypothesis/nested model)
delta_chi2 = chi2_lcdm - chi2_holo

# 2. Determine degrees of freedom difference
# Holographic has 2 extra parameters: mg and lambda_N
dof_diff = k_holo - k_lcdm

print(f"Chi2 (LambdaCDM):   {chi2_lcdm:.2f}")
print(f"Chi2 (Holographic): {chi2_holo:.2f}")
print(f"Delta Chi2:         {delta_chi2:.2f}")
print(f"Extra Params (DOF): {dof_diff}")

# 3. Calculate Significance (Wilks' Theorem)
if delta_chi2 <= 0:
    print("\nResult: The Holographic model does not improve the fit compared to LambdaCDM.")
    print("Significance: 0.00 σ")
else:
    # Calculate p-value: Probability of seeing this delta_chi2 by chance if LCDM were true
    p_value = stats.chi2.sf(delta_chi2, df=dof_diff)

    # Convert p-value to Gaussian Sigmas (two-tailed equivalent)
    # isf = Inverse Survival Function (Inverse 1-CDF)
    sigma_significance = stats.norm.isf(p_value / 2)

    print(f"\nP-value:      {p_value:.4f}")
    print(f"Significance: {sigma_significance:.2f} σ")

    if sigma_significance < 1:
        print("-> Conclusion: No statistically significant preference.")
    elif sigma_significance < 3:
        print("-> Conclusion: Mild preference (Tension), but not significant.")
    else:
        print("-> Conclusion: Strong evidence for Holographic model.")

## $f(R,\mathcal{G},T)+m_g$ massive gravity model.

 The MCMC sampling within the physically motivated range (-0.1,0.1) to test for deviations from standard matter conservation ($\beta=0$).


In [ ]:
from cobaya.run import run

info_frgt = {
    "likelihood": {
        "bao.desi_dr2": None,
        "sn.pantheon_plus": None
    },
    "theory": {
        "fRGT": {"external": "holographic_fRGT.py:HolographicFRGT"}
    },
    "params": {
        "H0": {"prior": {"min": 60, "max": 80}, "ref": 67.8},
        "Omega_m": {"prior": {"min": 0.1, "max": 0.5}, "ref": 0.31},
        "mg": {"prior": {"min": 0, "max": 0.5}, "ref": 0.12},
        "lambda_N": {"prior": {"min": 0, "max": 0.3}, "ref": 0.07},
        # Coupling parameter beta: beta=0 recovers the f(R,G) model
        "beta": {"prior": {"min": -0.1, "max": 0.1}, "ref": 0.0, "latex": r"\beta"}
    },
    "sampler": {
        "mcmc": {"max_samples": 2000, "Rminus1_stop": 0.02}
    },
    "output": "chains/fRGT_analysis"
}

updated_info_frgt, sampler = run(info_frgt)

## Relativistic Topological Quantum Field Theoretic (RTQFT)

topological correction $\chi_{topo}$ to $f(R,\mathcal{G},T) + m_g$ massive gravity model
using simulated DESI DR2 binned data

To determine if the DESI DR2 data can distinguish between the matter-curvature coupling ($\beta$
) and the RTQFT-inspired topological correction ($\chi_{topo}$
), use Neural Posterior Estimation (NPE).

In [ ]:
import torch
from sbi import utils as utils
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi

# 1. Define the Simulator (based on your H(z) definitions)
def simulator(params):
    # Unpack parameters: H0, Om, mg, lambdaN, beta, xi_topo
    H0, Om, mg, lN, beta, xi = params
    z = torch.tensor([0.1, 0.5, 1.0, 1.5, 2.0, 2.5]) # Example DESI bins

    # f(R,G,T) + RTQFT physics
    term_m = Om * (1 + z)**(3 - beta)
    term_de = (1 - Om) * (1 + lN * torch.log(1 + z)) * torch.exp(-mg * z)
    term_rtqft = xi * (1 + z)**4 * torch.exp(-1 / (1 + z))

    H_z = H0 * torch.sqrt(term_m + term_de + term_rtqft)
    # Return H(z) with added Gaussian noise to simulate DESI uncertainties
    return H_z + torch.randn(len(z)) * 0.5

# 2. Define Priors
# [H0, Om, mg, lN, beta, xi]
prior = utils.BoxUniform(low=torch.tensor([60., 0.1, 0., 0., -0.1, -0.05]),
                         high=torch.tensor([80., 0.5, 0.5, 0.3, 0.1, 0.05]))

# 3. Prepare for Neural Inference
simulator, prior = prepare_for_sbi(simulator, prior)
inference = SNPE(prior=prior)

# 4. Generate Training Data (e.g., 5000 simulations)
theta, x = simulate_for_sbi(simulator, proposal=prior, num_simulations=5000)
density_estimator = inference.append_simulations(theta, x).train()
posterior = inference.build_posterior(density_estimator)